In [ ]:
import os
import json
import numpy as np
from scipy import interpolate

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

#### Just the regular seaborn and matplotlib parameter tuning

In [ ]:
# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.1',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

## Loading train data

In [ ]:
tr_data_path = 'D:\\MS COCO\\train2014\\'

Collect image names

In [ ]:
tr_prefix = 'COCO_train2014_'
tr_suffix = '.jpg'
# Image ID
# Always 12 numbers, starting with zeros
tr_index = []
for name in os.listdir(tr_data_path):
    tr_index.append(int(name.split('_')[-1].split('.')[0]))

Load an example image

In [ ]:
def make_image_id(index):
    
    zeros = (12 - len(str(index))) * '0'
    return zeros + str(index)

In [ ]:
# Get random filename
tr_rand_index = make_image_id(np.random.choice(tr_index))
tr_expample = tr_data_path + tr_prefix + tr_rand_index + tr_suffix
# Load image
tr_example_img = plt.imread(tr_expample)

In [ ]:
tr_fig_norm = 40
fig, axes = plt.subplots(figsize=tuple(t/tr_fig_norm for t in tr_example_img.shape[:2]))

axes.imshow(tr_example_img)

plt.show()

## Load numerous samples

In [ ]:
n_samples = 5
tr_fig_norm = 40
fig, axes = plt.subplots(nrows=1, ncols=n_samples, figsize=(n_samples*8, 1*8))

for i in range(n_samples):
    # Get random filename
    tr_rand_index = make_image_id(np.random.choice(tr_index))
    tr_expample = tr_data_path + tr_prefix + tr_rand_index + tr_suffix
    # Load image
    tr_example_img = plt.imread(tr_expample)

    axes[i].imshow(tr_example_img)
    axes[i].axis('off')

plt.savefig('./documentation/tex/img/sample_images.png',
            bbox_inches='tight')
plt.show()

## Loading train labels

In [ ]:
label_path = 'D:\\MS COCO\\annotations\\'

In [ ]:
tr_caps_json = 'captions_train2014.json'
with open(label_path + tr_caps_json, 'r') as f:
    tr_caps = json.load(f)

In [ ]:
tr_labels_json = 'instances_train2014.json'
with open(label_path + tr_labels_json, 'r') as f:
    tr_labels = json.load(f)

In [ ]:
tr_caption_keys = list(tr_caps.keys())
tr_annot_keys = list(tr_labels.keys())

In [ ]:
print('Captions:', tr_caption_keys)
print('Labels:', tr_annot_keys)

In [ ]:
tr_caps['annotations'][0]

In [ ]:
#tr_labels['annotations'][0]

### Collect supercategories

In [ ]:
tr_labels['categories']

In [ ]:
super_cats = list(set([entry['supercategory'] for entry in tr_labels['categories']]))
sub_ids = {entry['id']: idx for idx, entry in enumerate(tr_labels['categories'])}

In [ ]:
super_cats

In [ ]:
def set_super_cat_colors(super_cats, cmap, vmin=0, vmax=1):
    
    assert vmin >= 0, "Parameter \'vmin\' should be >=0 !"
    assert vmax <= 1, "Parameter \'vmax\' should be <=1 !"
    
    cmap_values = np.linspace(vmin, vmax, len(super_cats))
    colors = {cat: cmap(value) for cat, value in zip(super_cats, cmap_values)}
    
    return colors

In [ ]:
super_cat_colors = set_super_cat_colors(super_cats=super_cats, cmap=cm.jet, vmin=0, vmax=1)

## Sorting annotation entries by image IDs

In [ ]:
tr_sorted_caps = sorted(tr_caps['annotations'], key=lambda x: x['image_id'], reverse=False)
tr_sorted_annot = sorted(tr_labels['annotations'], key=lambda x: x['image_id'], reverse=False)

tr_index_in_caps = [entry['image_id'] for entry in tr_sorted_caps]
tr_index_in_annot = [entry['image_id'] for entry in tr_sorted_annot]

## Load a random train image with annotations

Choose a random image ID

In [ ]:
tr_rand_index = np.random.choice(tr_index)
print('Random image ID: {0}'.format(tr_rand_index))

In [ ]:
tr_expample = tr_data_path + tr_prefix + make_image_id(tr_rand_index) + tr_suffix
# Load image
tr_example_img = plt.imread(tr_expample)

In [ ]:
tr_fig_norm = 40
fig, axes = plt.subplots(figsize=tuple(t/tr_fig_norm for t in tr_example_img.shape[:2]))

axes.imshow(tr_example_img)

plt.show()

Collect all entries from `tr_caps['annotations']` and `tr_labels['annotations']`, which is linked to the randomly chosen image

In [ ]:
tr_rand_caps_locs = np.where(tr_index_in_caps == tr_rand_index)[0]
tr_rand_annot_locs = np.where(tr_index_in_annot == tr_rand_index)[0]

In [ ]:
print(tr_rand_caps_locs)
print(tr_rand_annot_locs)

In [ ]:
tf_rand_caps = [tr_sorted_caps[idx] for idx in tr_rand_caps_locs]
tf_rand_annot = [tr_sorted_annot[idx] for idx in tr_rand_annot_locs]

In [ ]:
tr_fig_norm = 40
fig, axes = plt.subplots(figsize=tuple(t/tr_fig_norm for t in tr_example_img.shape[:2]))

axislegendsize = 15

axes.imshow(tr_example_img)

caption = 'Captions:'
for idx, entry in enumerate(tf_rand_caps):
    caption += '\n{0}. {1}'.format(idx+1, entry['caption'])
axes.set_title(caption)

for annot in tf_rand_annot:
    # Get category and supercategory
    cat_id = annot['category_id']
    super_cat = tr_labels['categories'][sub_ids[cat_id]]['supercategory']
    sub_cat = tr_labels['categories'][sub_ids[cat_id]]['name']
    color = super_cat_colors[super_cat]
    
    tr_rand_bbox = annot['bbox']
    rect = Rectangle(xy=tr_rand_bbox[:2], width=tr_rand_bbox[2], height=tr_rand_bbox[3],
                     lw=2, ec=color, fc='none', label=sub_cat)
    axes.add_patch(rect)

axes.legend(fontsize=axislegendsize,
            bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    
plt.show()